# Indexing and selecting data

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn
except ImportError:
    pass

In [4]:
# redefining the example objects

# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London


Setting the index to the country names:

In [5]:
countries = countries.set_index('country')
countries

,population,area,capital
country,,,
Belgium,11.3,30510,Brussels
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
Netherlands,16.9,41526,Amsterdam
United Kingdom,64.9,244820,London


## Some notes on selecting data

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. We now have to distuinguish between:

- selection by label
- selection by position.

### `data[]` provides some convenience shortcuts 

For a DataFrame, basic indexing selects the columns.

Selecting a single column:

In [6]:
countries['area']

country
Belgium            30510
France            671308
Germany           357050
Netherlands        41526
United Kingdom    244820
Name: area, dtype: int64

or multiple columns:

In [7]:
countries[['area', 'population']]

,area,population
country,,
Belgium,30510,11.3
France,671308,64.3
Germany,357050,81.3
Netherlands,41526,16.9
United Kingdom,244820,64.9


But, slicing accesses the rows:

In [8]:
countries['France':'Netherlands']

,population,area,capital
country,,,
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
Netherlands,16.9,41526,Amsterdam


<div class="alert alert-danger">
    <b>NOTE</b>: Unlike slicing in numpy, the end label is **included**.
</div>

So as a summary, `[]` provides the following convenience shortcuts:

- Series: selecting a label: `s[label]`
- DataFrame: selecting a single or multiple columns: `df['col']` or `df[['col1', 'col2']]`
- DataFrame: slicing the rows: `df['row_label1':'row_label2']` or `df[mask]`

### Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

These methods index the different dimensions of the frame:

* `df.loc[row_indexer, column_indexer]`
* `df.iloc[row_indexer, column_indexer]`

Selecting a single element:

In [9]:
countries.loc['Germany', 'area']

357050

But the row or column indexer can also be a list, slice, boolean array, ..

In [10]:
countries.loc['France':'Germany', ['area', 'population']]

,area,population
country,,
France,671308,64.3
Germany,357050,81.3


---
Selecting by position with `iloc` works similar as indexing numpy arrays:

In [11]:
countries.iloc[0:2,1:3]

,area,capital
country,,
Belgium,30510,Brussels
France,671308,Paris


The different indexing methods can also be used to assign data:

In [15]:
countries2 = countries.copy()
countries2.loc['Belgium':'Germany', 'population'] = 10

In [16]:
countries2

,population,area,capital
country,,,
Belgium,10.0,30510,Brussels
France,10.0,671308,Paris
Germany,10.0,357050,Berlin
Netherlands,16.9,41526,Amsterdam
United Kingdom,64.9,244820,London


## Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with 'boolean indexing' (like a where clause in SQL). 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [17]:
countries

,population,area,capital
country,,,
Belgium,11.3,30510,Brussels
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
Netherlands,16.9,41526,Amsterdam
United Kingdom,64.9,244820,London


In [20]:
countries['area']

country
Belgium            30510
France            671308
Germany           357050
Netherlands        41526
United Kingdom    244820
Name: area, dtype: int64

In [18]:
countries['area'] > 100000

country
Belgium           False
France             True
Germany            True
Netherlands       False
United Kingdom     True
Name: area, dtype: bool

In [19]:
countries[countries['area'] > 100000]

,population,area,capital
country,,,
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
United Kingdom,64.9,244820,London


---

<div class="alert alert-success">
    <b>EXERCISE</b>: Add a column `density` with the population density (note: population column is expressed in millions)
</div>

In [23]:
countries["density"] = countries["population"] * 1000000 / countries["area"]

<div class="alert alert-success">
    <b>EXERCISE</b>: Select the capital and the population column of those countries where the density is larger than 300
</div>

In [24]:
countries[countries["density"] > 300]["capital"]

country
Belgium         Brussels
Netherlands    Amsterdam
Name: capital, dtype: object

<div class="alert alert-success">
    <b>EXERCISE</b>: Add a column 'density_ratio' with the ratio of the density to the mean density
</div>

In [25]:
countries["density_ratio"] = countries["density"] / countries["density"].mean()

<div class="alert alert-success">
    <b>EXERCISE</b>: Change the capital of the UK to Cambridge
</div>

In [36]:
countries.loc["United Kingdom","capital"] = "Cambridge"


<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries whose population density is between 100 and 300 people/km²
</div>

In [40]:
countries[(countries["density"] > 100) & (countries["density"] < 300) ]

,population,area,capital,density,density_ratio
country,,,,,
Germany,81.3,357050,Berlin,227.699202,0.833502
United Kingdom,64.9,244820,Cambridge,265.092721,0.970382


## Some other useful methods: `isin` and string methods

The `isin` method of Series is very useful to select rows that may contain certain values:

In [42]:
s = countries['capital']

In [ ]:
s.isin?

In [43]:
s.isin(['Berlin', 'London'])

country
Belgium           False
France            False
Germany            True
Netherlands       False
United Kingdom    False
Name: capital, dtype: bool

This can then be used to filter the dataframe with boolean indexing:

In [44]:
countries[countries['capital'].isin(['Berlin', 'London'])]

,population,area,capital,density,density_ratio
country,,,,,
Germany,81.3,357050,Berlin,227.699202,0.833502


Let's say we want to select all data for which the capital starts with a 'B'. In Python, when having a string, we could use the `startswith` method:

In [45]:
'Berlin'.startswith('B')

True

In pandas, these are available on a Series through the `str` namespace:

In [46]:
countries['capital'].str.startswith('B')

country
Belgium            True
France            False
Germany            True
Netherlands       False
United Kingdom    False
Name: capital, dtype: bool

For an overview of all string methods, see: http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling

<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries that have capital names with more than 7 characters
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries that have capital names that contain the character sequence 'am'
</div>

In [ ]:
countries[countries['capital'].str.len() > 7]

## Pitfall: chained indexing (and the 'SettingWithCopyWarning')

In [70]:
countries.loc['Belgium', 'capital'] = 'Ghent' 

In [71]:
countries

,population,area,capital,density,density_ratio
country,,,,,
Belgium,11.3,30510,Ghent,370.370370,1.355755
France,64.3,671308,Paris,95.783158,0.350618
Germany,81.3,357050,Berlin,227.699202,0.833502
Netherlands,16.9,41526,Amsterdam,406.973944,1.489744
United Kingdom,64.9,244820,Cambridge,265.092721,0.970382


In [74]:
countries.loc['capital', 'Belgium'] = 'Antwerp' 

In [75]:
countries

,population,area,capital,density,density_ratio,Belgium
country,,,,,,
Belgium,11.3,30510.0,Antwerp,370.370370,1.355755,NaN
France,64.3,671308.0,Paris,95.783158,0.350618,NaN
Germany,81.3,357050.0,Berlin,227.699202,0.833502,NaN
Netherlands,16.9,41526.0,Amsterdam,406.973944,1.489744,NaN
United Kingdom,64.9,244820.0,Cambridge,265.092721,0.970382,NaN
capital,NaN,NaN,NaN,NaN,NaN,Antwerp


In [76]:
countries[countries['capital'] == 'Antwerp']['capital'] = 'Brussels' 

C:\envdev\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
countries.loc[countries['capital'] == 'Antwerp', 'capital'] = "Brussels"

How to avoid this?

* Use `loc` instead of chained indexing if possible!
* Or `copy` explicitly if you don't want to change the original data.

## More exercises!

For the quick ones among you, here are some more exercises with some larger dataframe with film data. These exercises are based on the [PyCon tutorial of Brandon Rhodes](https://github.com/brandon-rhodes/pycon-pandas-tutorial/) (so all credit to him!) and the datasets he prepared for that. You can download these data from here: [`titles.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKajNMa1pfSzN6Q3M) and [`cast.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKal9UYTJSR2ZhSW8) and put them in the `/data` folder.

In [96]:
cast = pd.read_csv('data/cast.csv')
cast.head()

,title,year,name,type,character,n
0,Suuri illusioni,1985,Homo $,actor,Guests,22.0
1,Gangsta Rap: The Glockumentary,2007,Too $hort,actor,Himself,NaN
2,Menace II Society,1993,Too $hort,actor,Lew-Loc,27.0
3,Porndogs: The Adventures of Sadie,2009,Too $hort,actor,Bosco,3.0
4,Stop Pepper Palmer,2014,Too $hort,actor,Himself,NaN


In [81]:
titles = pd.read_csv('data/titles.csv')
titles.head()

,title,year
0,The Rising Son,1990
1,Ashes of Kukulcan,2016
2,The Thousand Plane Raid,1969
3,Crucea de piatra,1993
4,The 86,2015


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies are listed in the titles dataframe?
</div>

In [85]:
# 215981.000000
titles.count()

title    215981
year     215981
dtype: int64

<div class="alert alert-success">
    <b>EXERCISE</b>: What are the earliest two films listed in the titles dataframe?
</div>

In [98]:
#titles[:].nsmallest(2,"year")
titles.loc[:].nsmallest(2,"year")

,title,year
165182,Miss Jerry,1894
85708,Reproduction of the Corbett and Fitzsimmons Fight,1897


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies have the title "Hamlet"?
</div>

In [111]:
#titles[titles["title"].str.startswith("Hamlet")].count()
#titles.loc[titles["title"].str.contains("Hamlet")].count()
#titles.loc[titles["title"] == "Hamlet"].count()
#len(titles[titles.title == 'Hamlet'])
len(titles.loc[titles.title == 'Hamlet'])


19

<div class="alert alert-success">
    <b>EXERCISE</b>: List all of the "Treasure Island" movies from earliest to most recent.
</div>

In [113]:
titles.loc[titles.title == 'Treasure Island'].sort_values("year")

,title,year
191379,Treasure Island,1918
47769,Treasure Island,1920
192917,Treasure Island,1934
90175,Treasure Island,1950
104714,Treasure Island,1972
103646,Treasure Island,1973
190792,Treasure Island,1985
166675,Treasure Island,1999


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies were made from 1950 through 1959?
</div>

In [126]:
#titles.loc[(titles.title == 'Treasure Island') & (titles.year >= 1950) & (titles.year <= 1959) ].sort_values("year")
#titles.loc[(titles.year >= 1950) & (titles.year <= 1959) ].count()
len(titles.loc[(titles.year >= 1950) & (titles.year <= 1959) ])

12120

<div class="alert alert-success">
    <b>EXERCISE</b>: How many roles in the movie "Inception" are NOT ranked by an "n" value?
</div>

In [136]:
#len(cast.loc[ (cast["title"]=="Inception") & (pd.isna(cast["n"])) ])
len(cast.loc[ (cast["title"]=="Inception") & (cast.n.isnull()) ])


,title,year,name,type,character,n
86093,Inception,2010,Michael (III) August,actor,LAX Passenger,NaN
174676,Inception,2010,Ken Bhan,actor,Mombasan Gambler,NaN
333474,Inception,2010,John Ceallach,actor,LAX Passenger,NaN
610905,Inception,2010,Seve Esposito,actor,Italian Gambler,NaN
631993,Inception,2010,Zachary Christopher Fay,actor,Extra,NaN
747030,Inception,2010,Daniel Girondeaud,actor,Bridge Sub Con,NaN
900363,Inception,2010,Andrew Hoagland,actor,Pedestrian,NaN
979629,Inception,2010,Charles Jarrell,actor,Hotel Guest,NaN
991536,Inception,2010,Seong-hwan Jo,actor,Japanese Landing Crew,NaN
1001735,Inception,2010,Alonzo F. Jones,actor,Mombasan Casino Staff,NaN


<div class="alert alert-success">
    <b>EXERCISE</b>: But how many roles in the movie "Inception" did receive an "n" value?
</div>

In [138]:
len(cast.loc[(cast["title"] == "Inception") & (cast.n.notnull()) ])

51

<div class="alert alert-success">
    <b>EXERCISE</b>: Display the cast of "North by Northwest" in their correct "n"-value order, ignoring roles that did not earn a numeric "n" value.
</div>

In [144]:
cast.loc[(cast["title"] == "North by Northwest") & cast.n.notnull() ].sort_values("n")

,title,year,name,type,character,n
778278,North by Northwest,1959,Cary Grant,actor,Roger O. Thornhill,1.0
3104905,North by Northwest,1959,Eva Marie Saint,actress,Eve Kendall,2.0
1300582,North by Northwest,1959,James Mason,actor,Phillip Vandamm,3.0
2794420,North by Northwest,1959,Jessie Royce Landis,actress,Clara Thornhill,4.0
317310,North by Northwest,1959,Leo G. Carroll,actor,The Professor,5.0
2702399,North by Northwest,1959,Josephine Hutchinson,actress,Mrs. Townsend,6.0
1513974,North by Northwest,1959,Philip Ober,actor,Lester Townsend,7.0
1137555,North by Northwest,1959,Martin Landau,actor,Leonard,8.0
2181363,North by Northwest,1959,Adam Williams,actor,Valerian,9.0
1616877,North by Northwest,1959,Edward Platt,actor,Victor Larrabee,10.0


<div class="alert alert-success">
    <b>EXERCISE</b>: How many roles were credited in the silent 1921 version of Hamlet?
</div>

In [145]:
len(cast.loc[(cast["title"] == "Hamlet") & (cast.year == 1921)])

9

<div class="alert alert-success">
    <b>EXERCISE</b>: List the supporting roles (having n=2) played by Cary Grant in the 1940s, in order by year.
</div>

In [149]:
cast.loc[(cast.name == "Cary Grant") & ( (cast.year // 10) == 194) & (cast.n == 2)].sort_values("year")

,title,year,name,type,character,n
778275,My Favorite Wife,1940,Cary Grant,actor,Nick,2.0
778285,Penny Serenade,1941,Cary Grant,actor,Roger Adams,2.0
